<a href="https://colab.research.google.com/github/mifm/examples/blob/master/hopp_hydesign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install hydesign
!pip install hopp

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd drive/My Drive

/content/drive/My Drive


In [8]:
import os
os.makedirs('tool-benchmarking', exist_ok=True)
!git clone https://github.com/IEAWindTask50/tool-benchmarking.git

Cloning into 'tool-benchmarking'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 111 (delta 39), reused 94 (delta 28), pack-reused 0 (from 0)
Receiving objects: 100% (111/111), 7.30 MiB | 11.81 MiB/s, done.
Resolving deltas: 100% (39/39), done.


In [9]:
%cd tool-benchmarking/Europe/

/content/drive/MyDrive/tool-benchmarking/Europe/tool-benchmarking/Europe


In [10]:
import time
import numpy as np
import pandas as pd
import os
from hydesign.assembly.hpp_assembly import hpp_model

In [15]:
longitude = -0.864258
latitude = 48.744116
altitude = 302
input_ts_fn = './hyd_input/input_ts_France_good_wind.csv'
sim_pars_fn = './hyd_input/hpp_pars.yml'

hpp = hpp_model(
        latitude=latitude,
        longitude=longitude,
        altitude=altitude,
        num_batteries = 10,
        work_dir = './',
        sim_pars_fn = sim_pars_fn,
        input_ts_fn = input_ts_fn,
)


Fixed parameters on the site
-------------------------------
longitude = -0.864258
latitude = 48.744116
altitude = 302

input_ts_fn length is not a complete number of years (hyDesign handles years as 365 days).
The file has been modified and stored in ./input_ts_modified.csv


In [16]:
input_ts_fn = './input_ts_modified.csv'
sim_pars_fn = './hyd_input/hpp_pars.yml'

hpp = hpp_model(
        latitude=latitude,
        longitude=longitude,
        altitude=altitude,
        num_batteries = 10,
        work_dir = './',
        sim_pars_fn = sim_pars_fn,
        input_ts_fn = input_ts_fn,
)

inputs = dict(
clearance =	60,
sp =	287.6391934,
p_rated =	10,
Nwt	=	25,
wind_MW_per_km2 =	4.782274215,
solar_MW =	223.2191223,
surface_tilt =	50,
surface_azimuth = 210,
DC_AC_ratio	=1.295598428,
b_P =75.27402422,
b_E_h=	4.451612903,
cost_of_battery_P_fluct_in_peak_price_ratio	=0.580645161,)


Fixed parameters on the site
-------------------------------
longitude = -0.864258
latitude = 48.744116
altitude = 302


In [17]:
start = time.time()
outs = hpp.evaluate(**inputs)
hpp.print_design([v for k,v in inputs.items()], outs)
end = time.time()
print('exec. time [min]:', (end - start)/60 )
print(hpp.prob['NPV_over_CAPEX'])


Design:
---------------
clearance [m]: 60.000
sp [W/m2]: 287.639
p_rated [MW]: 10.000
Nwt: 25.000
wind_MW_per_km2 [MW/km2]: 4.782
solar_MW [MW]: 223.219
surface_tilt [deg]: 50.000
surface_azimuth [deg]: 210.000
DC_AC_ratio: 1.296
b_P [MW]: 75.274
b_E_h [h]: 4.452
cost_of_battery_P_fluct_in_peak_price_ratio: 0.581


NPV_over_CAPEX: 0.133
NPV [MEuro]: 57.149
IRR: 0.072
LCOE [Euro/MWh]: 33.649
Revenues [MEuro]: 48.052
CAPEX [MEuro]: 429.258
OPEX [MEuro]: 6.900
Wind CAPEX [MEuro]: 275.586
Wind OPEX [MEuro]: 5.599
PV CAPEX [MEuro]: 65.197
PV OPEX [MEuro]: 1.301
Batt CAPEX [MEuro]: 21.810
Batt OPEX [MEuro]: 0.000
Shared CAPEX [MEuro]: 66.665
Shared OPEX [MEuro]: 0.000
penalty lifetime [MEuro]: 0.000
AEP [GWh]: 1202.856
GUF: 0.458
grid [MW]: 300.000
wind [MW]: 250.000
solar [MW]: 223.219
Battery Energy [MWh]: 335.091
Battery Power [MW]: 75.274
Total curtailment [GWh]: 56.301
Total curtailment with deg [GWh]: 124.902
Awpp [km2]: 52.276
Apvp [km2]: 2.737
Plant area [km2]: 52.276
Rotor diam [m]

In [18]:
from hopp.simulation import HoppInterface

/content/drive/MyDrive/tool-benchmarking/Europe/tool-benchmarking/Europe/log/hybrid_systems_2025-06-20T18.26.14.843627.log


In [19]:
import yaml
with open('hopp_input/france.yaml', 'r') as f:
  france_yaml = yaml.load(f, Loader=yaml.FullLoader)

france_yaml['solar_resource_file'] = '../../tool-benchmarking/Europe/hopp_input/france.csv'
france_yaml['wind_resource_file'] = '../../tool-benchmarking/Europe/hopp_input/france.srw'
france_yaml['grid_resource_file'] = '../../tool-benchmarking/Europe/hopp_input/france-price.csv'

with open('hopp_input/france.yaml', 'w') as f:
  yaml.dump(france_yaml, f)

In [20]:
hi = HoppInterface(os.path.join(os.getcwd(), "hopp_input/08-wind-solar-france.yaml"))


FLORIS is the system model...


KeyError: 'power_thrust_data_file'